---
Author: Mustapha Bouhsen <br>
[LinkedIn](https://www.linkedin.com/in/mustapha-bouhsen/)<br>
[Git](https://github.com/mus514)<br>
Date: February 2, 2024<br>
---


### Load files from Azure blob storage : Set the data location and type


In [0]:
%run Repos/bouhsen.m@gmail.com/ML_Pipeline_Hub/library/daily_utilities

In [0]:
# storage_account_name = "lemusdata"
# storage_key = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-04-01T01:53:24Z&st=2024-02-13T18:53:24Z&spr=https&sig=uuyzcpnt9q%2B%2FZhq%2FA2Vk3A14a7P8hrERo8kk149ocag%3D"

# container_name = "tables"
# mount_point = "/mnt/tables"

# dbutils.fs.mount(
#   source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
#   mount_point = mount_point,
#   extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net":storage_key})

True

In [0]:
# Delete mnt
# dbutils.fs.unmount(mount_point)

In [0]:
#-----------------------------------------
# Set the raw and the prod folder paths
#-----------------------------------------
raw_folder_path = "/mnt/raw/"
clean_folder_path = "/mnt/clean/"

raw_files_paths = [file.path for file in dbutils.fs.ls(raw_folder_path)]

In [0]:
#-----------------------------------------
# The schema
#-----------------------------------------
schema = StructType([
    StructField("date", DateType(), True),
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True),
    StructField("adj_close", FloatType(), True),
    StructField("volume", FloatType(), True)
])

col_float = ["open", "high", "low", "close", "volume"]

In [0]:
def process_stocks(raw_files_paths):
    try:
        for file_path in raw_files_paths:
            stock_name = file_path.split("/")[-1].split(".")[0]
            print(f'process start for {stock_name}')
            # Load the data
            df = spark.read.option("header", "True").schema(schema).csv(file_path)

            # Temp folder to save temp parquet files
            temp_folder = clean_folder_path + f"temp/"

            # the file name path
            file_name_path = temp_folder + f"/{stock_name}/"
            
            # write data frame to csv
            df.write.mode("overwrite").option("header", "True").csv(file_name_path)

            # get all files path ending with .parquet
            files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
            # Copy parquet files to final destination
            ingest_and_transform_to_parquet(files_paths, clean_folder_path, stock_name)

            # delete the temp folder
            delete_contents_recursively(temp_folder)
            print(f'process end for {stock_name}')
            print('--------------------------------')  

    except Exception as e:
        print("The error is: ",e)
    


In [0]:
process_stocks(raw_files_paths)

process start for AAPL
process end for AAPL
--------------------------------
process start for AMZN
process end for AMZN
--------------------------------
process start for GOOGL
process end for GOOGL
--------------------------------
process start for MSFT
process end for MSFT
--------------------------------
